# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
- Difference between Classification and Regression :
  - **Regression learning** *has a continuous output (one real value)*. The output is the goal, the thing we try to find by the learning process. Regression could be represented by a linear model with this form : ax+b (where 'a' is the slope of the line and 'b', his intercept. For example, if we try to find the age of persons with their picture as input, the representation should be a linear model where each person (his picture could be represented on the graph abscisse by his name) will be mapped with the corresponding age (on the graph ordinate).
  - **Classification learning** *has a binary or discrete output, {0, 1, ..., n}*. According to the type of problem to solve, these output could be given by labeled statements. For example if we have to determine if a person is a "person of interest", the output should be "TRUE" or "FALSE".
- **In our case**, the outputs (the values we try to find by this learning process) are discrete. Indeed, we try to determine if a student would fail according to some information on him as input, and provide assistance if the prediction is positive. So the output should be for example "FAILURE" or "SUCCESS". So, it is clear that we have here a **Classification learning problem**.


## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully! \n"
#print student_data
#print student_data['passed']
#print student_data.size
#print len(student_data.columns)


Student data read successfully! 



### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
"""
nbn=0
nbp=0
for elt in student_data['passed']:
    if elt=='no':
        nbn=nbn+1
    if elt=='yes':
        nbp=nbp+1

print "Number of n_passed ", nbp
print "Number of n_failed ", nbn
"""

# TODO: Calculate number of students
n_students = len(student_data)#None

# TODO: Calculate number of features
features = student_data.drop('passed', axis = 1)
n_features = len(features.columns)#None

# TODO: Calculate passing students

n_passed = len(student_data[student_data['passed'] == 'yes']['passed'])#None

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no']['passed'])#None

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/float(n_students))*100 #None

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
#print len(feature_cols)
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [8]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# Our dataset is small and slightly unbalanced. (There are more passing students than on passing students)
# So to obtain stratification (consitence) we use stratify = y_all (on the label, our prediction goal)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify = y_all, test_size=95, random_state=42)

# TODO: Set the number of training points
num_train = 300 #None

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = X_train[:num_train] #None
X_test = X_test[:num_test] #None
y_train = y_train #None
y_test = y_test #None

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
Here are the supervised learning models that, I think, are appropriate for this problem :
- **The *"Decision Tree"* model**
   - *What is it ?* : Decision Tree is a ***non-parametric*** supervised learning method used for classification and regression. It is used to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. The rules are simple conditional statements (for example, if-then-else decision rules). 
     
   - *Strengths* : 
     - Simple to understand and to interpret. Trees can be visualised.
     - Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed.
     - Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable.
     - Possible to validate a model using statistical tests.
     - Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.
   - *Weaknesses* : 
     - Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. However it is possible to use the pruning method (prune the tree or ensemble many trees in a random forest ) to remove nodes that do not provide additional information. Pruning should reduce the size of a learning tree without reducing predictive accuracy as measured by a cross-validation set.
     - Decision trees can be unstable because small variations in the data might result in a completely different tree being generated.
     - There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
     - Decision tree learners create biased trees if some classes dominate.
   - *Why is it appropriate for this problem ?* : 
     - It is appropriate for our current problem because we are in a case of classification supervised learning. We learn from an existing sample data that already contains inputs and ouputs values. And we want to predict the value of a target variable which determine if a student passed or not. So as it does, Decision Tree will help us predict the value of our target variable. Moreover the Decision Tree model requires little data preparation, and is simple to understand and to interpret. So it will allow us to work more easily on our data. In addition, we could generate understandable graphs, and with methods like *export_graphviz* we could visualize trees with their nodes (attributes) and parameter's values.
- **The *"Support Vector Machine (SVM)"* model**
   - *What is it ?* : Support Vector Machine (SVM) is a set of supervised learning methods used for classification, regression and outliers detection.   
   - *Strengths* : 
     - Effective in high dimensional spaces.
     - Still effective in cases where number of dimensions is greater than the number of samples.
     - Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
     - Versatile: different Kernel functions can be specified for the decision function.

   - *Weaknesses* : 
     - As any other classification algorithm, if the number of features is much greater than the number of samples, the method is likely to give poor performances.
     - SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation.
   - *Why is it appropriate for this problem ?* : 
     - It is appropriate for our current problem because we are in a case of classification supervised learning. Moreover with SVM we could easily detect outliers values. For example we can easily detect students with abnormal parameters value (very low or very high). On another hand, we have about 50 features and because SVM is effective in high dimensional spaces, we are sure to have reliable estimations.
- **The *"Naive Bayes"* model**
   - *What is it ?* : Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ theorem with the “naive” assumption of independence between every pair of features. More clearly, Naive Bayes  algorithms apply Bayes's probabilistic conditionnal independance on the features to give some predictions on data, as if there was an independence between every pair of features. It is used to solve classification problems.
To perform his goal Naive Bayes uses several classifiers according to the cases :
       - *GaussianNB* : implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian.
       - *MultinomialNB* : implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification
       - *BernoulliNB* : implements the naive Bayes training and classification algorithms for data that is distributed according to multivariate Bernoulli distributions; i.e., there may be multiple features but each one is assumed to be a binary-valued (Bernoulli, boolean) variable.
       - *Out-of-core naive Bayes model fitting* : Naive Bayes models can be used to tackle large scale classification problems for which the full training set might not fit in memory. To handle this case, MultinomialNB, BernoulliNB, and GaussianNB expose a partial_fit method that can be used incrementally.
   - *Strengths* : 
      - Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods
      - Each feature distribution can be independently estimated as a one dimensional distribution (this helps to alleviate problems stemming from the curse of dimensionality).
      - Inference is cheap (simplicity).
      - Uses few parameters (no hyper-parameters).
      - Estimate parameters with labeled data.
   - *Weaknesses* : 
      - It is a bad estimator. So the probability outputs from *predict_proba* are not to be taken too seriously. The model is likely to correctly assign the highest probability to the most likely class but this probability is unlikely to be accurate. This is why it is highly recommended to always work with optimal parameters with Naive Bayes.
   - *Why is it appropriate for this problem ?* : 
     - It is appropriate for our current problem because we are in a case of classification supervised learning. Moreover with Naive Bayes methods we could easily evaluate each feature separetly as if we was in a one dimension case, using classifier's attributes like *theta_* (to know the mean of each feature per class) and *sigma_* (variance of each feature per class). So it will be very useful if we want more information about a particular feature. Another important reason is that, as we know that Naive Bayes learners and classifiers are easy and cheap to train (extremely fast), we could use them based on their low requirement of computational resources. Naive Bayes can run easily and fastly on any kind of computer.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [9]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [10]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.tree import DecisionTreeClassifier
# from sklearn import model_B
from sklearn import svm
# from skearln import model_C
from sklearn.naive_bayes import GaussianNB

# TODO: Initialize the three models
# The random state allows to obtain reproducible results.
clf_A = DecisionTreeClassifier(random_state=42) #None
clf_B = svm.SVC(random_state=42) #None
clf_C = GaussianNB() #None

# TODO: Set up the training set sizes
"""
X_train_100 = X_train[:100] #None
y_train_100 = y_train[:100] #None

X_train_200 = X_train[:200] #None
y_train_200 = y_train[:200] #None

X_train_300 = X_train[:300] #None
y_train_300 = y_train[:300] #None


# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

# train_predict for model_A, DecisionTree
print " ===================== model_A, DecisionTree ==================== "
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

# train_predict for model_B, SVM
print " ===================== model_B, SVM ==================== "
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

# train_predict for model_C, NAIVE BAYES
print " ===================== model_C, NAIVE BAYES ==================== "
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
"""

for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: ================================= \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)



DecisionTreeClassifier: ================================= 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6452.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0000 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7258.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0000 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6838.

SVC: ================================= 

Training a SVC using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8354.
Made prediction

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0020          |        0.0000          |      1.0000      |     0.6452      |
| 200               |         0.0020          |        0.0000          |      1.0000      |     0.7258      |
| 300               |         0.0030          |        0.0010          |      1.0000      |     0.6838      |

** Classifer 2 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0020          |        0.0010          |      0.8354      |     0.8025      |
| 200               |         0.0060          |        0.0040          |      0.8431      |     0.8105      |
| 300               |         0.0100          |        0.0080          |      0.8664      |     0.8052      |

** Classifer 3 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0010          |        0.0010          |      0.7752      |     0.6457      |
| 200               |         0.0010          |        0.0010          |      0.8060      |     0.7218      |
| 300               |         0.0020          |        0.0010          |      0.8134      |     0.7761      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** According to the previous experiment I performed on the available data, I will propose the best model to choose after a presentation of the times and scores performances analysis :

*** Times Performance analysis ***
- All the models (DecisionTreeClassifier, SVM and GaussianNB) have approximatively a zero (0.0000 s) prediction time.
- We also see that for DecisionTreeClassifier and GaussianNB, their training time turns arround to zero (0.0000 s) when the training set size increase. This means that if we use them on a big data set they will train all the data fastly.
- But the training time of the SVM classifier (SVC) increases with the quantity of data. Therefore, it grows slowly when the data increases. From a data set of 200 students to 300 students, we note a variation of about 2.17 % on the training time (mean value of several measurements).
For a data set of 300 students, the training time is about 0.047 s. This implies that , on this basis, we need about two seconds to train a data set of 10,000 students (with Scikit learn, SVM can only handle roughly up to 10,000 samples), and we need 156 to 200 seconds (about 3 minutes) to train a data set of one million students (to reach this level we should train a SVM online with Pegasos or other stochastic gradient decent algorithm).

*** Scores Performance analysis ***
- The Decision Tree algorithm produces the worst score performances. There is an important gap between the training and test scores (arround 0.3), even when the data increases. This is a sign of high variance and could lead to overfitting. So, with its actual parameters, this model will not well generalizes to unseen data. It should need more tunning process for improvement. 
- SVM and GaussianNB give the best score performances. For the both, the training and test scores are good and closer when the data increases (the gap is less than 0.1). And their scores seem to converge to a constant value when the data increases (arround 0.8).
But the SVM algorithm produces the highest training and test scores, compared to the GaussianNB.

*** Final proposal ***
- According to the times and scores analysis above, I recommend to choose the SVM algorithm as it is the the most appropriate based on the available data, limited resources, cost, and performance. Indeed, even if SVM has a slow training time compared to others, the rate of 3 minutes to train a data set of one million students is very acceptable. Generally, schools have more less students a year than this number, so the training time should be less in reality. Another possibility is that, as the SVM algorithm must hold the entire training set in memory during the training process, increase the memory capacity of the computer should also reduce considerably the training time. And ***once the data are trained, all the predictions are done very fastly***. Moreover, the essential thing is that ***SVM already produces the best scores for training and test data. So it is already ready to well generalizes to unseen data, and give reliable prediction on the plausible failure or success of a given student***. This is a sign of the fact that SVM naturally support well the structure of the data. We have just to tune it again to improve more its performance. We also know that Naive Bayes has the weakness to be a bad estimator, so I definitely propose SVM. After this, the other option should be the tuning of the Decision Tree model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
- The SVM algorithm will be implemented in a file to produce the prediction. First we will take the raw students data and fill it in the program file. Then the data will be reorganized into two types : the feature data and the target data. The feature data will contain all information about the caracteristics of the students. And the target data will contain the information about what we want to predict (the failure or success of a given student).
From these two prepared types of data we will generate suitable data sets that can be fitted in the SVM algorithm for test and prediction implementation. Only the prepared data sets could be used to make prediction.
- From the prepared data sets we will test the SVM prediction scores according to its own parameters until to find the best parameters that produces the best scores. The best scores determine that the model will well generalizes to unseen data after the final implementation. So how does SVM work ? It will separe the data into two classes according to the data points caracteristics : the students who have succeeded and those who have failed. And so it will finally define a margin, the maximal distance between the two classes. Then, the SVM will determine a line in the margin center as a decision boundary. All the data point on the side of the plan where the failure class is will be considered as a failure, and those on the other side of the plan will be considered as a success.
And to well perform theses operations, the SVM need to be runned with the best parameters we mentionned previously.Once we have theses best parameters for SVM, we will implement the final version with them. So when the file will be applied to unseen data, it will automatically make reliable prediction about the students results, according to the information in the data. The program file execution will generate the expected information about students: their plausible failure or success.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [62]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer

# TODO: Create the parameters list you wish to tune
parameters = { 'C' : [ 1, 10, 100, 1000 ], 'gamma' : [ 0.1, 10, 100 ] } # Some
#parameters = { 'C' : [ 100 ], 'gamma' : [ 0.1 ] } # Some

# TODO: Initialize the classifier
clf = svm.SVC() #None

# TODO: Make an f1 scoring function using 'make_scorer' 
# score = make_scorer(my_custom_loss_func, greater_is_better=True)
f1_scorer = make_scorer(f1_score, pos_label='yes') #None


# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
#grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer) #None
# Pro Tip: I apply here the pro tip with use of a stratified shuffle split data-split which 
# preserves the percentage of samples for each class and combines it with cross validation.

#SVC parameters
# 1. Let's build a stratified shuffle object
ssscv = StratifiedShuffleSplit( y_train, n_iter=100, test_size=0.1) 
# 2. Let's now we pass the object and the parameters to grid search
grid_obj = GridSearchCV(clf, parameters, cv = ssscv , scoring=f1_scorer) 

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train) #None

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print " ================= "
print " The best estimators for the model : \n", clf.get_params()
print " ================= "


Made predictions in 0.0160 seconds.
Tuned model has a training F1 score of 0.9781.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8258.
 The best estimators for the model : 
{'kernel': 'rbf', 'C': 1, 'verbose': False, 'probability': False, 'degree': 3, 'shrinking': True, 'max_iter': -1, 'decision_function_shape': None, 'random_state': None, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'gamma': 0.1, 'class_weight': None}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
- The final model's F1 score for training is **0.9781**
- The final model's F1 score for testing is **0.8258**
- Compared to the untuned model which has a training score of 0.8664 and a testing score of 0.8052, we note clearly that the scores values have increased with the tunned model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In [61]:
# ADDITIONAL Personnal prediction test on data
# As you could verify, once the data are trained, all the predictions are done very fastly.

# Show predictions
print " =============== PREDICTED GRADUATE STATE ================= "
for i, graduate_state in enumerate(clf.predict(X_train[:20])):
    print "Would Student {} succeed ? : {}".format(i+1, graduate_state)

 =============== PREDICTED GRADUATE STATE ================= 
Would Student 1 succeed ? : yes
Would Student 2 succeed ? : yes
Would Student 3 succeed ? : yes
Would Student 4 succeed ? : no
Would Student 5 succeed ? : yes
Would Student 6 succeed ? : no
Would Student 7 succeed ? : no
Would Student 8 succeed ? : yes
Would Student 9 succeed ? : no
Would Student 10 succeed ? : yes
Would Student 11 succeed ? : yes
Would Student 12 succeed ? : yes
Would Student 13 succeed ? : yes
Would Student 14 succeed ? : yes
Would Student 15 succeed ? : yes
Would Student 16 succeed ? : yes
Would Student 17 succeed ? : yes
Would Student 18 succeed ? : yes
Would Student 19 succeed ? : yes
Would Student 20 succeed ? : yes
